#  XGBoost

## Instalation

PIP: pip install --user xgboost

CPU only: conda install -c conda-forge py-xgboost-cpu

Use NVIDIA GPU: conda install -c conda-forge py-xgboost-gpu

## Introducing XGBoost

Boosting, especially of decision trees, is among the most prevalent and powerful machine learning algorithms.

There are many variants of boosting algorithms and frameworks implementing those algorithms. XGBoost — short for extreme gradient boosting — is one of the most well-known algorithms with an accompanying, and even more popular, framework.

As the name may reveal, XGBoost is a gradient boosting algorithm, a common technique in ensemble learning. Ensemble learning is a type of machine learning that enlists many models to make predictions together. 

Boosting algorithms are distinguished from other ensemble learning techniques by building a sequence of initially weak models into increasingly more powerful models. Gradient boosting algorithms choose how to build a more powerful model using the gradient of a loss function that captures the performance of a model.

XGBoost operates on decision trees, models that construct a graph that examines the input under various “if” statements (vertices in the graph). Whether the “if” condition is satisfied influences the next “if” condition and eventual prediction. XGBoost progressively adds more and more “if” conditions to the decision tree to build a stronger model.

## An example - XGBoost regression

### Loading and exploring the data

We will be working with the Diamonds dataset throughout the tutorial. It is built into the Seaborn library. It has a nice combination of numeric and categorical features and over 50k observations that we can comfortably showcase all the advantages of XGBoost.

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

diamonds = sns.load_dataset("diamonds")

diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


We will just look at the 5-number summary of the numeric and categorical features and get going. You can spend a few moments to familiarize yourself with the dataset.

In [2]:
diamonds.shape

(53940, 10)

In [3]:
diamonds.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [4]:
diamonds.describe(exclude=np.number)

,cut,color,clarity
count,53940,53940,53940
unique,5,7,8
top,Ideal,G,SI1
freq,21551,11292,13065


### Build an XGBoost DMatrix

After you are done with exploration, the first step in any project is framing the machine learning problem and extracting the feature and target arrays based on the dataset.

We will first try to predict diamond prices using their physical measurements, so our target will be the `price` column.

So, we are isolating the features into `X` and the target into `y`:

In [6]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

The dataset has three categorical columns. Normally, you would encode them with ordinal or one-hot encoding, but XGBoost has the ability to internally deal with categoricals.

The way to enable this feature is to cast the categorical columns into Pandas category data type (by default, they are treated as text columns):

In [8]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

X.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
x           float64
y           float64
z           float64
dtype: object

Let’s split the data into train, and test sets (0.25 test size):

In [9]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Now, the important part: XGBoost comes with its own class for storing datasets called DMatrix. It is a highly optimized class for memory and speed. That's why converting datasets into this format is a requirement for the native XGBoost API:

In [10]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

The class accepts both the training features and the labels. To enable automatic encoding of Pandas category columns, we also set enable_categorical to True.

After building the DMatrices, you should choose a value for the `objective` parameter. It tells XGBoost the machine learning problem you are trying to solve and what metrics or loss functions to use to solve that problem.

For example, to predict diamond prices, which is a regression problem, you can use the common `reg:squarederror` objective. Usually, the name of the objective also contains the name of the loss function for the problem. For regression, it is common to use Root Mean Squared Error.

> A note on the difference between a loss function and a performance metric: A loss function is used by machine learning models to minimize the *differences* between the actual (ground truth) values and model predictions. On the other hand, a metric (or metrics) is chosen by the machine learning engineer to measure the *similarity* between ground truth and model predictions.

### Training

The chosen objective function and any other hyperparameters of XGBoost should be specified in a dictionary, which by convention should be called params:

In [11]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist"}

Now, we set another parameter called `num_boost_round`, which stands for number of boosting rounds. Internally, XGBoost minimizes the loss function RMSE in small incremental rounds (more on this later). This parameter specifies the amount of those rounds.

The ideal number of rounds is found through hyperparameter tuning. For now, we will just set it to 100:

In [12]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg, #our train matrix
   num_boost_round=n,
)

### Evaluation

During the boosting rounds, the model object has learned all the patterns of the training set it possibly can. Now, we must measure its performance by testing it on unseen data. That's where our `dtest_reg` DMatrix comes into play:

In [15]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)
rmse = mean_squared_error(y_test, preds, squared=False)

print(f"RMSE of the base model: {round(rmse, 3)}")

RMSE of the base model: 555.607


We’ve got a base score ~555$, which was the performance of a base model with default parameters. There are two ways we can improve it — by performing cross-validation and hyperparameter tuning. But before that, let’s see a quicker way of evaluating XGBoost models.

### Using Validation Sets During Training

Training a machine learning model is like launching a rocket into space. You can control everything about the model up to the launch, but once it does, all you can do is stand by and wait for it to finish.

But the problem with our current training process is that we can’t even watch where the model is going. To solve this, we will use evaluation arrays that allow us to see model performance as it gets improved incrementally across boosting rounds.

Let’s create a list of two tuples that each contain two elements. The first element is the array for the model to evaluate, and the second is the array’s name.

In [17]:
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

When we pass this array to the `evals` parameter of `xgb.train`, we will see the model performance after each boosting round:

In [18]:
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
)

[0]	train-rmse:2874.29379	validation-rmse:2817.38773
[1]	train-rmse:2092.07711	validation-rmse:2054.73630
[2]	train-rmse:1549.52687	validation-rmse:1526.30592
[3]	train-rmse:1184.46798	validation-rmse:1174.90119
[4]	train-rmse:941.09127	validation-rmse:943.28272
[5]	train-rmse:784.58014	validation-rmse:796.09651
[6]	train-rmse:685.75110	validation-rmse:705.22245
[7]	train-rmse:624.67281	validation-rmse:653.32563
[8]	train-rmse:584.19599	validation-rmse:620.30404
[9]	train-rmse:558.77667	validation-rmse:599.24504
[10]	train-rmse:543.85303	validation-rmse:586.99790
[11]	train-rmse:531.92694	validation-rmse:578.68120
[12]	train-rmse:523.08456	validation-rmse:571.73527
[13]	train-rmse:515.67753	validation-rmse:567.19913
[14]	train-rmse:510.77594	validation-rmse:564.66402
[15]	train-rmse:506.68519	validation-rmse:563.21547
[16]	train-rmse:502.96796	validation-rmse:561.80880
[17]	train-rmse:498.90184	validation-rmse:560.36561
[18]	train-rmse:492.74859	validation-rmse:558.46274
[19]	train-rms

You can see how the model minimizes the score after each round.

In real-world projects, you usually train for thousands of boosting rounds, which means that many rows of output. To reduce them, you can use the `verbose_eval` parameter, which forces XGBoost to print performance updates every `vebose_eval` rounds:

In [19]:
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10 # Every ten rounds
)

[0]	train-rmse:2874.29379	validation-rmse:2817.38773
[10]	train-rmse:543.85303	validation-rmse:586.99790
[20]	train-rmse:487.42071	validation-rmse:556.44229
[30]	train-rmse:460.86396	validation-rmse:554.68339
[40]	train-rmse:444.03762	validation-rmse:552.62130
[50]	train-rmse:430.07110	validation-rmse:553.50718
[60]	train-rmse:418.57995	validation-rmse:555.44368
[70]	train-rmse:406.77489	validation-rmse:555.06703
[80]	train-rmse:394.18070	validation-rmse:555.00800
[90]	train-rmse:382.65353	validation-rmse:555.74725
[99]	train-rmse:373.74308	validation-rmse:555.60692


### XGBoost Early Stopping

By now, you must have realized how important boosting rounds are. Generally, the more rounds there are, the more XGBoost tries to minimize the loss. But this doesn’t mean the loss will always go down. Let’s try with 3500 boosting rounds with the verbosity of 500:

In [21]:
n = 3500

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=500
)

[0]	train-rmse:2874.29379	validation-rmse:2817.38773
[500]	train-rmse:197.72375	validation-rmse:563.29248
[1000]	train-rmse:121.69016	validation-rmse:572.18689
[1500]	train-rmse:84.17960	validation-rmse:576.42698
[2000]	train-rmse:60.88001	validation-rmse:578.58142
[2500]	train-rmse:46.47975	validation-rmse:579.96944
[3000]	train-rmse:36.59124	validation-rmse:580.78514
[3499]	train-rmse:29.49081	validation-rmse:581.36713


We get the lowest loss before round 500. After that, even though training loss keeps going down, the validation loss (the one we care about) keeps increasing.

When given an unnecessary number of boosting rounds, XGBoost starts to overfit and memorize the dataset. This, in turn, leads to validation performance drop because the model is memorizing instead of generalizing.

Remember, we want the golden middle: a model that learned just enough patterns in training that it gives the highest performance on the validation set. So, how do we find the perfect number of boosting rounds, then?

We will use a technique called early stopping. Early stopping forces XGBoost to watch the validation loss, and if it stops improving for a specified number of rounds, it automatically stops training.

This means we can set as high a number of boosting rounds as long as we set a sensible number of early stopping rounds.

For example, let’s use 10000 boosting rounds and set the `early_stopping_rounds` parameter to 50. This way, XGBoost will automatically stop the training if validation loss doesn't improve for 50 consecutive rounds.

In [22]:
n = 10000

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2874.29379	validation-rmse:2817.38773
[50]	train-rmse:430.07110	validation-rmse:553.50718
[89]	train-rmse:384.05170	validation-rmse:555.95636


### XGBoost Cross-Validation

At the beginning we set aside 25% of the dataset for testing. The test set would allow us to simulate the conditions of a model in production, where it must generate predictions for unseen data.

But only a single test set would not be enough to measure how a model would perform in production accurately. For example, if we perform hyperparameter tuning using only a single training and a single test set, knowledge about the test set would still “leak out.” How?

Since we try to find the best value of a hyperparameter by comparing the validation performance of the model on the test set, we will end up with a model that is configured to perform well only on that particular test set. Instead, we want a model that performs well across the board — on any test set we throw at it.

The solution is cross-validation. In cross-validation, we still have two sets: training and testing.

While the test set waits in the corner, we split the training into 3, 5, 7, or `k` splits or folds. Then, we train the model `k` times. Each time, we use `k-1` parts for training and the final `k`th part for validation. This process is called k-fold cross-validation.

After all folds are done, we can take the mean of the scores as the final, most realistic performance of the model.

Let’s perform this process in code using the `cv` function of XGB:

In [31]:
n = 1000

results = xgb.cv(
   params, dtrain_reg,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)

In [34]:
results.head(10)

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2874.530912,9.576510,2877.437274,37.093540
1,2089.327469,8.317290,2094.021636,24.828795
2,1550.617973,5.223297,1558.386252,18.540267
3,1183.812759,5.193420,1195.032441,13.471580
4,941.203113,4.539805,958.728828,9.479449
5,784.505805,5.707896,807.806455,9.228214
6,684.295140,4.217592,711.731819,9.488339
7,620.605084,3.991168,652.864677,10.612015
8,581.944546,4.668284,617.502530,10.319193
9,556.851294,4.102871,596.822911,11.646375


In [33]:
best_rmse = results['test-rmse-mean'].min()
best_rmse

549.311480649509

## An example 2 - XGBoost Classification

Building an XGBoost classifier is as easy as changing the objective function; the rest can stay the same.

The two most popular classification objectives are:

* `binary:logistic` - binary classification (the target contains only two classes, i.e., cat or dog)
* `multi:softprob` - multi-class classification (more than two classes in the target, i.e., apple/orange/banana)

Performing binary and multi-class classification in XGBoost is almost identical, so we will go with the latter. Let’s prepare the data for the task first.

We want to predict the cut quality of diamonds given their price and their physical measurements. So, we will build the feature/target arrays accordingly.

The only difference is that since XGBoost only accepts numbers in the target, we are encoding the text classes in the target with `OrdinalEncoder` of Sklearn.

In [35]:
from sklearn.preprocessing import OrdinalEncoder

X, y = diamonds.drop("cut", axis=1), diamonds[['cut']]

# Encode y to numeric
y_encoded = OrdinalEncoder().fit_transform(y)

# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to pd.Categorical
for col in cats:
   X[col] = X[col].astype('category')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=1, stratify=y_encoded)

Now, we build the DMatrices...

In [36]:
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

…and set the objective to `multi:softprob`. This objective also requires the number of classes to be set by us (`cut` has 5 unique values, so the `num_class` param will be 5):

In [38]:
params = {"objective": "multi:softprob", "tree_method": "hist", "num_class": 5}
n = 100

results = xgb.cv(
   params, dtrain_clf,
   num_boost_round=n,
   nfold=5,
   metrics=["mlogloss", "auc", "merror"],
)

During cross-validation, we are asking XGBoost to watch three classification metrics which report model performance from three different angles. Here is the result:

During cross-validation, we are asking XGBoost to watch three classification metrics which report model performance from three different angles. Here is the result:

In [39]:
results.keys()

Index(['train-mlogloss-mean', 'train-mlogloss-std', 'train-auc-mean',
       'train-auc-std', 'train-merror-mean', 'train-merror-std',
       'test-mlogloss-mean', 'test-mlogloss-std', 'test-auc-mean',
       'test-auc-std', 'test-merror-mean', 'test-merror-std'],
      dtype='object')

For example, to see the best AUC score, we take the maximum of test-auc-mean column:

In [40]:
results['test-auc-mean'].max()

0.9387768185864125